# Python 知识

## 元类

### type

Python 有个内置类型 `type`，其是所有类的类型（注意实例的类型是类）。

`type` 有两个作用：
- 查看对象类型：`type(obj)`
- 动态创建类 `type(name, bases, dict)`
  - `name（字符串）`：新类的名称。
  - `bases（元组）`：新类需要继承的父类组成的元组。
  - `dict（字典）`：包含属性和方法。键是属性或方法的名称（字符串），字值是对应的属性值或函数对象。

In [ ]:
def fn(self, name='world'):
    print('Hello, %s.' % name)

Hello = type('Hello', (object,), dict(hello=fn))
h = Hello()
h.hello()

### `*()` 等价于 `type(*).__call__`

注意 `type` 有方法 `__call__`：先调用 `__new__` 再调用 `__init__`。

### 类创建过程

当 Python 遇到类定义时，例如
```python
class MyClass(BaseClass, metaclass=MyMeta):
    attr = "value"
    
    def method(self):
        pass
```
会经历以下步骤：
- 收集信息
  - 类名：`MyClass`
  - 基类：`(BaseClass,)`
  - 命名空间：`{'attr': 'value', 'method': <function>}`
  - 元类：`MyMeta`
    - 元类查找顺序
      - 显式指定的 `metaclass` 参数
      - 基类的元类 
      - 默认的 `type`
- 调用元类：`   MyClass = MyMeta('MyClass', (BaseClass,), {'attr': 'value', 'method': <function>})`

In [ ]:
class DetailedMeta(type):
    def __call__(cls, *args, **kwargs):
        print(f"\n=== DetailedMeta.__call__ ===")
        print(f"正在创建类的实例")
        print(f"类: {cls}")
        print(f"参数: {args}, {kwargs}")
        
        # 当创建类的实例时调用这里
        instance = super().__call__(*args, **kwargs)
        print(f"实例创建完成: {instance}")
        return instance
    
    def __new__(mcs, name, bases, namespace):
        print(f"\n=== DetailedMeta.__new__ ===")
        print(f"正在创建类: {name}")
        print(f"元类: {mcs}")
        print(f"基类: {bases}")
        print(f"原始命名空间: {list(namespace.keys())}")
        
        # 修改命名空间
        namespace['_created_by_meta'] = True
        namespace['_meta_name'] = mcs.__name__
        
        print(f"修改后命名空间: {list(namespace.keys())}")
        
        # 创建类对象
        cls = super().__new__(mcs, name, bases, namespace)
        print(f"类对象创建完成: {cls}")
        
        return cls
    
    def __init__(cls, name, bases, namespace):
        print(f"\n=== DetailedMeta.__init__ ===")
        print(f"正在初始化类: {cls}")
        print(f"类名: {name}")
        
        # 添加类级别的属性
        cls._creation_info = {
            'name': name,
            'bases': [base.__name__ for base in bases],
            'methods': [key for key in namespace.keys() 
                       if callable(namespace[key]) and not key.startswith('__')]
        }
        
        super().__init__(name, bases, namespace)

# 基类
class BaseClass:
    def base_method(self):
        return "base method"

# 现在让我们观察创建过程
print("开始创建类...")
MyClass = DetailedMeta('MyClass', (BaseClass,), {
    'attr': 'value',
    'my_method': lambda self: "my method result"
})

print(f"\n创建的类: {MyClass}")
print(f"类的属性: {dir(MyClass)}")
print(f"创建信息: {MyClass._creation_info}")

# 创建实例
print(f"\n开始创建实例...")
obj = MyClass()
print(f"实例: {obj}")

### 例子：自定义单例模式元类

In [ ]:
class SingletonMeta(type):
    """单例模式元类"""
    _instances = {}
    
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

class Database(metaclass=SingletonMeta):
    def __init__(self):
        self.connection = "Connected to DB"

# 测试单例模式
db1 = Database()
db2 = Database()
print(db1 is db2)  # True - 同一个实例

### 例子：使用函数作为元类

In [ ]:
def debug_meta(name, bases, attrs):
    """调试元类：打印类创建信息"""
    print(f"创建类: {name}")
    print(f"基类: {bases}")
    print(f"属性: {list(attrs.keys())}")
    
    # 为所有方法添加调试装饰器
    for key, value in attrs.items():
        if callable(value) and not key.startswith('__'):
            attrs[key] = debug_wrapper(value)
    
    return type(name, bases, attrs)

def debug_wrapper(func):
    def wrapper(*args, **kwargs):
        print(f"调用方法: {func.__name__}")
        return func(*args, **kwargs)
    return wrapper

class Calculator(metaclass=debug_meta):
    def add(self, a, b):
        return a + b
    
    def multiply(self, a, b):
        return a * b

# 输出：
# 创建类: Calculator
# 基类: ()
# 属性: ['__module__', '__qualname__', 'add', 'multiply']

calc = Calculator()
result = calc.add(2, 3)  # 调用方法: add

# 代码分析

## 类继承关系
```mermaid
classDiagram
    %% 元类层次结构
    class type {
        <<metaclass>>
        +__new__(mcs, name, bases, namespace)
        +__init__(cls, name, bases, namespace)  
        +__call__(cls, *args, **kwargs)
    }
    
    class MetaPlotsBuilderMixin {
        <<metaclass>>
        +subplots(cls) Config
    }
    
    %% 主要类
    class PlotsBuilderMixin {
        +_subplots : Config
        -__init__()
        +writeable_attrs : Set[str]
        +plots_defaults : dict
        +subplots : Config
        +plots() Optional[BaseFigure]
        +build_subplots_doc(source_cls) str
        +override_subplots_doc(__pdoc__, source_cls) None
    }
    
    %% 继承关系
    type <|-- MetaPlotsBuilderMixin
    MetaPlotsBuilderMixin <|-- PlotsBuilderMixin : metaclass
```

## class MetaPlotsBuilderMixin(type)

``` python
class MetaPlotsBuilderMixin(type):
    @property
    def subplots(cls) -> Config:
        return cls._subplots
```

## class PlotsBuilderMixin(metaclass=MetaPlotsBuilderMixin)

被某类所继承，该类必须同时还继承 `Wrapping` 并且定义 `_subplots`，例如：

### 使用示例

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from vectorbt.base.array_wrapper import ArrayWrapper, Wrapping
from vectorbt.generic.plots_builder import PlotsBuilderMixin
from vectorbt.utils.config import Config


class StockAnalyzer(Wrapping, PlotsBuilderMixin):
    # 定义子图配置 - 使用PlotsBuilderMixin的核心
    _subplots = Config({
        # 价格图表
        'price': {
            'title': '股价走势图',
            'plot_func': 'plot_price',  # 对应的绘图方法名
            'tags': ['basic', 'price', 'ohlc'],  # 标签用于过滤
            'yaxis_kwargs': {'title': '价格 (元)'},
            'xaxis_kwargs': {'title': '日期'}
        },
        
        # 成交量图表
        'volume': {
            'title': '成交量',
            'plot_func': 'plot_volume',
            'tags': ['basic', 'volume'],
            'yaxis_kwargs': {'title': '成交量'},
            'xaxis_kwargs': {'title': '日期'}
        },
        
        # 移动平均线
        'moving_averages': {
            'title': '移动平均线',
            'plot_func': 'plot_moving_averages',
            'tags': ['technical', 'trend'],
            'yaxis_kwargs': {'title': '价格 (元)'},
            'xaxis_kwargs': {'title': '日期'}
        },
        
        # RSI相对强弱指标
        'rsi': {
            'title': 'RSI 相对强弱指标',
            'plot_func': 'plot_rsi',
            'tags': ['technical', 'oscillator'],
            'yaxis_kwargs': {'title': 'RSI', 'range': [0, 100]},
            'xaxis_kwargs': {'title': '日期'}
        },
        
        # MACD指标
        'macd': {
            'title': 'MACD 指标',
            'plot_func': 'plot_macd',
            'tags': ['technical', 'trend'],
            'yaxis_kwargs': {'title': 'MACD'},
            'xaxis_kwargs': {'title': '日期'}
        },
        
        # 布林带
        'bollinger_bands': {
            'title': '布林带',
            'plot_func': 'plot_bollinger_bands',
            'tags': ['technical', 'volatility'],
            'yaxis_kwargs': {'title': '价格 (元)'},
            'xaxis_kwargs': {'title': '日期'}
        }
    })
    
    def __init__(self, data: pd.DataFrame, symbol: str = "STOCK"):
        """
        初始化股票分析器
        
        Args:
            data: 包含OHLCV数据的DataFrame，列名应包括：
                  ['Open', 'High', 'Low', 'Close', 'Volume']
            symbol: 股票代码或名称
        """
        # 验证数据格式
        required_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
        missing_columns = [col for col in required_columns if col not in data.columns]
        if missing_columns:
            raise ValueError(f"数据缺少必要的列: {missing_columns}")
        
        # 创建ArrayWrapper - 这是使用Wrapping基类的要求
        wrapper = ArrayWrapper.from_obj(data)
        
        # 调用父类初始化
        # 注意：必须先初始化Wrapping，再初始化PlotsBuilderMixin
        Wrapping.__init__(self, wrapper)
        PlotsBuilderMixin.__init__(self)
        
        # 存储数据和参数
        self._data = data
        self._symbol = symbol
        
        # 计算技术指标
        self._calculate_indicators()
    
    def _calculate_indicators(self):
        """计算各种技术指标"""
        # 移动平均线
        self._ma5 = self._data['Close'].rolling(window=5).mean()
        self._ma10 = self._data['Close'].rolling(window=10).mean()
        self._ma20 = self._data['Close'].rolling(window=20).mean()
        
        # RSI计算
        self._rsi = self._calculate_rsi(self._data['Close'])
        
        # MACD计算
        self._macd, self._macd_signal, self._macd_histogram = self._calculate_macd(self._data['Close'])
        
        # 布林带计算
        self._bb_upper, self._bb_middle, self._bb_lower = self._calculate_bollinger_bands(self._data['Close'])
    
    def _calculate_rsi(self, prices: pd.Series, period: int = 14) -> pd.Series:
        """计算RSI指标"""
        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi
    
    def _calculate_macd(self, prices: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9):
        """计算MACD指标"""
        ema_fast = prices.ewm(span=fast).mean()
        ema_slow = prices.ewm(span=slow).mean()
        macd = ema_fast - ema_slow
        macd_signal = macd.ewm(span=signal).mean()
        macd_histogram = macd - macd_signal
        return macd, macd_signal, macd_histogram
    
    def _calculate_bollinger_bands(self, prices: pd.Series, period: int = 20, std_dev: int = 2):
        """计算布林带"""
        middle = prices.rolling(window=period).mean()
        std = prices.rolling(window=period).std()
        upper = middle + (std * std_dev)
        lower = middle - (std * std_dev)
        return upper, middle, lower
    
    # 绘图方法 - 这些方法会被PlotsBuilderMixin自动调用
    
    def plot_price(self, fig, row=None, col=None, **kwargs):
        """绘制价格K线图"""
        # 添加K线图
        fig.add_trace(
            go.Candlestick(
                x=self._data.index,
                open=self._data['Open'],
                high=self._data['High'],
                low=self._data['Low'],
                close=self._data['Close'],
                name=f'{self._symbol} K线',
                showlegend=True
            ),
            row=row, col=col
        )
    
    def plot_volume(self, fig, row=None, col=None, **kwargs):
        """绘制成交量柱状图"""
        # 根据涨跌设置颜色
        colors = ['red' if close >= open else 'green' 
                 for close, open in zip(self._data['Close'], self._data['Open'])]
        
        fig.add_trace(
            go.Bar(
                x=self._data.index,
                y=self._data['Volume'],
                name='成交量',
                marker_color=colors,
                showlegend=True
            ),
            row=row, col=col
        )
    
    def plot_moving_averages(self, fig, row=None, col=None, **kwargs):
        """绘制移动平均线"""
        # MA5
        fig.add_trace(
            go.Scatter(
                x=self._data.index,
                y=self._ma5,
                mode='lines',
                name='MA5',
                line=dict(color='blue', width=1),
                showlegend=True
            ),
            row=row, col=col
        )
        
        # MA10
        fig.add_trace(
            go.Scatter(
                x=self._data.index,
                y=self._ma10,
                mode='lines',
                name='MA10',
                line=dict(color='orange', width=1),
                showlegend=True
            ),
            row=row, col=col
        )
        
        # MA20
        fig.add_trace(
            go.Scatter(
                x=self._data.index,
                y=self._ma20,
                mode='lines',
                name='MA20',
                line=dict(color='red', width=1),
                showlegend=True
            ),
            row=row, col=col
        )
    
    def plot_rsi(self, fig, row=None, col=None, **kwargs):
        """绘制RSI指标"""
        fig.add_trace(
            go.Scatter(
                x=self._data.index,
                y=self._rsi,
                mode='lines',
                name='RSI',
                line=dict(color='purple', width=2),
                showlegend=True
            ),
            row=row, col=col
        )
        
        # 添加超买超卖线
        fig.add_hline(y=70, line_dash="dash", line_color="red", 
                     annotation_text="超买线(70)", row=row, col=col)
        fig.add_hline(y=30, line_dash="dash", line_color="green", 
                     annotation_text="超卖线(30)", row=row, col=col)
    
    def plot_macd(self, fig, row=None, col=None, **kwargs):
        """绘制MACD指标"""
        # MACD线
        fig.add_trace(
            go.Scatter(
                x=self._data.index,
                y=self._macd,
                mode='lines',
                name='MACD',
                line=dict(color='blue', width=2),
                showlegend=True
            ),
            row=row, col=col
        )
        
        # 信号线
        fig.add_trace(
            go.Scatter(
                x=self._data.index,
                y=self._macd_signal,
                mode='lines',
                name='Signal',
                line=dict(color='red', width=1),
                showlegend=True
            ),
            row=row, col=col
        )
        
        # 柱状图
        colors = ['red' if val >= 0 else 'green' for val in self._macd_histogram]
        fig.add_trace(
            go.Bar(
                x=self._data.index,
                y=self._macd_histogram,
                name='Histogram',
                marker_color=colors,
                showlegend=True
            ),
            row=row, col=col
        )
        
        # 零轴线
        fig.add_hline(y=0, line_dash="dash", line_color="gray", row=row, col=col)
    
    def plot_bollinger_bands(self, fig, row=None, col=None, **kwargs):
        """绘制布林带"""
        # 上轨
        fig.add_trace(
            go.Scatter(
                x=self._data.index,
                y=self._bb_upper,
                mode='lines',
                name='布林上轨',
                line=dict(color='red', width=1),
                showlegend=True
            ),
            row=row, col=col
        )
        
        # 中轨（移动平均线）
        fig.add_trace(
            go.Scatter(
                x=self._data.index,
                y=self._bb_middle,
                mode='lines',
                name='布林中轨',
                line=dict(color='blue', width=1),
                showlegend=True
            ),
            row=row, col=col
        )
        
        # 下轨
        fig.add_trace(
            go.Scatter(
                x=self._data.index,
                y=self._bb_lower,
                mode='lines',
                name='布林下轨',
                line=dict(color='green', width=1),
                fill='tonexty',  # 填充到上一条线
                fillcolor='rgba(0,100,80,0.1)',
                showlegend=True
            ),
            row=row, col=col
        )
    
    # 便捷属性访问器
    @property
    def data(self) -> pd.DataFrame:
        """获取原始数据"""
        return self._data
    
    @property
    def symbol(self) -> str:
        """获取股票代码"""
        return self._symbol
    
    @property
    def rsi(self) -> pd.Series:
        """获取RSI指标"""
        return self._rsi
    
    @property
    def macd(self) -> tuple:
        """获取MACD指标"""
        return self._macd, self._macd_signal, self._macd_histogram


def create_sample_data(symbol: str = "AAPL", days: int = 100) -> pd.DataFrame:
    """
    创建示例股票数据
    
    Args:
        symbol: 股票代码
        days: 数据天数
        
    Returns:
        包含OHLCV数据的DataFrame
    """
    # 创建日期索引
    dates = pd.date_range(start='2024-01-01', periods=days, freq='D')
    
    # 生成模拟价格数据
    np.random.seed(42)  # 确保结果可重现
    
    # 基础价格
    base_price = 100.0
    
    # 生成价格序列（随机游走）
    returns = np.random.normal(0.001, 0.02, days)  # 日收益率
    prices = [base_price]
    
    for i in range(1, days):
        new_price = prices[-1] * (1 + returns[i])
        prices.append(max(new_price, 1.0))  # 确保价格不为负
    
    # 生成OHLC数据
    data = []
    for i, price in enumerate(prices):
        # 生成当日波动
        daily_volatility = np.random.uniform(0.005, 0.03)
        high = price * (1 + daily_volatility)
        low = price * (1 - daily_volatility)
        
        # 开盘价（基于前一日收盘价）
        if i == 0:
            open_price = price
        else:
            open_price = prices[i-1] * (1 + np.random.normal(0, 0.01))
        
        # 收盘价
        close_price = price
        
        # 成交量（随机生成）
        volume = np.random.randint(1000000, 10000000)
        
        data.append({
            'Open': open_price,
            'High': max(open_price, high, close_price),
            'Low': min(open_price, low, close_price),
            'Close': close_price,
            'Volume': volume
        })
    
    df = pd.DataFrame(data, index=dates)
    return df


def main():
    # 1. 创建示例数据
    print("1. 创建示例股票数据...")
    stock_data = create_sample_data("AAPL", 100)
    print(f"数据形状: {stock_data.shape}")
    print(f"数据列: {list(stock_data.columns)}")
    print(f"日期范围: {stock_data.index[0]} 到 {stock_data.index[-1]}\n")
    
    # 2. 创建分析器实例
    print("2. 创建股票分析器...")
    analyzer = StockAnalyzer(stock_data, "AAPL")
    print(f"分析器类型: {type(analyzer)}")
    print(f"可用子图: {list(analyzer.subplots.keys())}")
    print(f"包装器形状: {analyzer.wrapper.shape}\n")
    
    # 3. 基本绘图示例
    print("3. 基本绘图示例...")
    
    # 绘制所有子图
    print("   a) 绘制所有子图")
    fig_all = analyzer.plots(subplots=['price', 'volume', 'rsi', 'macd'])

    return analyzer, {
        'all': fig_all,
    }


if __name__ == "__main__":
    analyzer, figures = main()
    figures['all'].show()

### `def __init__(self)`
```python
def __init__(self):
    checks.assert_instance_of(self, Wrapping)
    self._subplots = self.__class__._subplots.copy()
```

### writeable_attrs
参考类 `Configured`。
```python
@property
def writeable_attrs(self) -> tp.Set[str]:
    return {'_subplots'}
```

### plots_defaults
```python
@property
def plots_defaults(self) -> tp.Kwargs:
    from vectorbt._settings import settings  # 导入vectorbt全局设置
    plots_builder_cfg = settings['plots_builder']  # 获取图表构建器配置

    return merge_dicts(
        plots_builder_cfg,  # 全局默认配置
        dict(settings=dict(freq=self.wrapper.freq))  # 添加当前对象的频率信息
    )
```

### subplots
```python
@property
def subplots(self) -> Config:
    return self._subplots
```

### build_subplots_doc
```python
@classmethod
def build_subplots_doc(cls, source_cls: tp.Optional[type] = None) -> str:
    if source_cls is None:
        source_cls = PlotsBuilderMixin
    return string.Template(
        inspect.cleandoc(get_dict_attr(source_cls, 'subplots').__doc__)
    ).substitute(
        {'subplots': cls.subplots.to_doc(), 'cls_name': cls.__name__}
    )
```

### override_subplots_doc
```python
@classmethod
def override_subplots_doc(cls, __pdoc__: dict, source_cls: tp.Optional[type] = None) -> None:
    __pdoc__[cls.__name__ + '.subplots'] = cls.build_subplots_doc(source_cls=source_cls)
```